In [ ]:
import torch
import numpy as np

from raytorch.structures import learnable_meshes
from raytorch.ops.ray_intersect import test_print, ray_triangle_intersect_iter
from raytorch.LiDAR import LiDAR_base
from raytorch.visualization import visualize_LiDAR, visualize_point_clouds

from pytorch3d.utils import ico_sphere
from pytorch3d.vis.plotly_vis import plot_batch_individually, plot_scene
from pytorch3d.transforms import Translate



In [ ]:
ball = ico_sphere(level = 1)
translate = Translate(x=0, y=10, z=0)

obj = learnable_meshes(ball)

In [ ]:
lidar = LiDAR_base(torch.tensor([0.0, 0.0, 0.0]),
                    azi_range = [0, 180],
                    polar_range = [80, 100],
                    res = [2.5, 2.5],
                    min_range = 1.5,
                    max_range = 20)

In [ ]:
intersection = lidar.scan_triangles(obj.get_deformed_meshes(translate))
print(intersection)
plot_scene({
    "test": {"ray": visualize_LiDAR(lidar),
             "points": visualize_point_clouds(intersection)},
},
    raybundle_ray_line_width=2.0,
    pointcloud_marker_size=2.0
)

In [ ]:
intersection_single_ray = lidar.scan_triangles(obj.get_deformed_meshes(translate),
                                               method="single_ray")
print(intersection_single_ray)

intersection_iter = lidar.scan_triangles(obj.get_deformed_meshes(translate),
                                               method="iter")
print(intersection_iter)

In [ ]:
grad_input = torch.rand(intersection.size())

parameter = obj.get_parameters()
intersection_iter.backward(grad_input)
grad_iter = obj.get_gradient()

parameter.grad.zero_()
intersection_single_ray.backward(grad_input)
grad_single = obj.get_gradient()

print(f"is allclose: {torch.allclose(grad_iter, grad_single)}")